# Методы первого порядка

In [1]:
import numpy as np
import numpy.linalg as lg

## Данная плоская функция (+ мои тестовые)

In [2]:
def f_(x):
    return 0.5 * x ** 7 - x ** 3 + 0.5 * x ** 2 - x

def g_(x):
    return x ** 2

def h_(x):
    return 0.5 * x ** 6 - x ** 3 + 0.5 * x ** 2 - x


## "Умные" производные (плоский случай)

In [3]:
def der1(f, e):
    def der(x):
        return (f(x + e) - f(x)) / e
    return der

def der2(f, e):
    df = der1(f, e)
    def der2(x):
        return (df(x + e) - df(x)) / e
    return der2

## Метод касательных

In [4]:
def tangent(f, a_, b_, e):
    a = a_
    b = b_
    df = der1(f, e)
    if not(f(a) >= 0 and f(b) <= 0):
        c = ( f(b) - f(a) + df(a) * a - df(b) * b) / ( df(a) - df(b) )
        while abs(b - a) > e and df(c) > e and abs(f(b) - f(a)) > e:
            if df(c) > 0:
                b = c
            else:
                a = c
            c = ( f(b) - f(a) + df(a) * a - df(b) * b ) / ( df(a) - df(b) )
    return c

In [5]:
a = 1
b = 1.5
e = 0.05
x_min = tangent(f_, a, b, e)
print(x_min, f_(x_min))

1.0506541346069636 -0.9518829159541643


## Метод Ньютона-Рафсона

In [6]:
def Newton_Rafson(f, x, e):
    
    df = der1(f, e)
    while e < abs(df(x)):
        d2f = der2(f, e)
        x = x - df(x) / d2f(x)
        df = der1(f, e)
    return x

In [7]:
x0 = 3
e = 0.05
x_min = Newton_Rafson(f_, x0, e)
print(x_min, f_(x_min))

0.9406581908773792 -1.0047378588390956


## Метод секущих

In [8]:
def secant(f, x, e):
    x_ = x;
    e = 0.05;
    df = der1(f, e)
    if e < abs(df(x)):
        d2f = der2(f, e)
        x = x - df(x) / d2f(x)
        
        while  abs(df(x)) < e:
            x = x - (x - x_) / (df(x) - df(x_)) * df(x)
    return x

In [9]:
x0 = 1;
xi_ = x0;
x_min = secant(f_, x0, e)
print(x_min, f_(x_min))

0.9563258220120603 -1.0078901348763478


## Данная многомерня функция

In [10]:
def fx(x): 
    return np.exp(sum(np.square(x)))
def gradfx(x):
    return fx(x) * (2*x) 
def grad2fx(x):
    return fx(x) * 2 * (np.eye(len(x)) + 2 * x.reshape(len(x),1) * x)

#Сжатая относительно оси Ox1 функция f(x) 
def hx(x): 
    x1 = x
    x1[0] = x[0]/10
    return np.exp(sum(np.square(x1)))
def gradhx(x_):
    x = x_
    x[0] = x[0]/10
    return fx(x) * (2*x) 
def grad2hx(x_):
    x = x_
    x[0] = x[0]/10
    return fx(x) * 2 * (np.eye(len(x)) + 2 * x.reshape(len(x),1) * x)

#альтернативная ДВУМЕРНАЯ функция ( для проверки)
def gx(x):
    k = np.array([3, 1])
    return sum(np.square(k * x))
def gradgx(x):
    k = np.array([18, 2])
    return x * k
def grad2gx(x):
    return np.array([[18 ,0],[0, 2]])


## "Умные" производные (многомерный случай)

In [11]:
def func(f, x0, gradx0):
    def fun(a):
        A = x0 -  gradx0 * a
        return f(A)
    return fun

def der1_e(f_, x0, a, gradx0, e):
    f_(x0)
    f = func(f_, x0, gradx0)
    def der(a):
        return (f(a + e) - f(a)) / e
    return der

def der2_e(f_, x0, a, gradx0, e):
    df = der1_e(f_, x0, a, gradx0, e)
    def der2(a):
        return (df(a + e) - df(a))/e
    return der2

## Метод покоординатного спуска

In [12]:
def coord(_f, grad, x, e):
    print('')
    i = 0
    n = 0
    
    while True:
        n += 1 
        print('')
        print('------[----', n, '----]------')
        
        zero = np.zeros(len(x))
        zero[i] = 1
        
        f = func(_f, x, zero)
        a = Newton_Rafson(f, x, e)
        x_1 = x
        x = x -  zero * a
        print('  i  = ', i)
        print('  x  = ', x)
        print('f(x) = ', f(x))
        
        i = (i + 1) % len(x)
        
        if abs(_f(x) - _f(x_1)) < e and abs(lg.norm(x - x_1)) < e:
            break
        
    print('сделано ', n, ' шагов')
    return x



In [13]:
x0 = np.array([-1.0, 0.3, 1.])
e = 0.01
x_min = coord(fx,gradfx, x0, e)
print('answer:')
print(round(fx(x_min), 3), x_min)

x0 = np.array([-1.0, 0.3])
e = 0.01
x_min = coord(gx,gradgx, x0, e)
print('answer:')
print(round(gx(x_min), 3), x_min)

x0 = np.array([-1.0, 0.3, 0.4])
e = 0.01
x_min = coord(hx,gradhx, x0, e)
print('answer:')
print(round(hx(x_min), 3), x_min)



------[---- 1 ----]------
  i  =  0
  x  =  [0.0049985 0.3       1.       ]
f(x) =  8.166349452318341
grad =  [0.02973456 1.78460903 5.94869677]

------[---- 2 ----]------
  i  =  1
  x  =  [0.0049985 0.0049985 1.       ]
f(x) =  2.965515472680092
grad =  [0.02717602 0.02717602 5.43683533]

------[---- 3 ----]------
  i  =  2
  x  =  [0.0049985 0.0049985 0.0049985]
f(x) =  2.691444274270289
grad =  [0.00999775 0.00999775 0.00999775]

------[---- 4 ----]------
  i  =  0
  x  =  [0.0049985 0.0049985 0.0049985]
f(x) =  1.0000499712585194
grad =  [0.00999775 0.00999775 0.00999775]
сделано  4  шагов
answer:
1.0 [0.0049985 0.0049985 0.0049985]


------[---- 1 ----]------
  i  =  0
  x  =  [0.005 0.3  ]
f(x) =  9.180224999999998
grad =  [0.09 0.6 ]

------[---- 2 ----]------
  i  =  1
  x  =  [0.005 0.005]
f(x) =  0.08724999999999998
grad =  [0.09 0.01]

------[---- 3 ----]------
  i  =  0
  x  =  [0.005 0.005]
f(x) =  2.5000000000000045e-05
grad =  [0.09 0.01]
сделано  3  шагов
answer:
0.0